In [ ]:
#conda install -c conda-forge ggplot

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import nltk
import re
import gc

from datetime import datetime, timedelta
from plotnine import *
from pandas.api.types import CategoricalDtype
#from plotnine.data import mpg
#%matplotlib inline


In [3]:
import psutil
psutil.virtual_memory()

svmem(total=17179869184, available=9483493376, percent=44.8, used=7695474688, free=4253011968, active=5230911488, inactive=3310448640, wired=2464563200)

# Functions

In [61]:
# Create variables representing the week date range and week index in which the job was posted
def week_posted(df):
    """Create a variable representing the week in which the job was posted"""
    conditions_date = [
        (df['Date Posted'] >= datetime(2020, 3, 4)) & (df['Date Posted'] <= datetime(2020, 3, 7)),
        (df['Date Posted'] >= datetime(2020, 3, 8)) & (df['Date Posted'] <= datetime(2020, 3, 14)),
        (df['Date Posted'] >= datetime(2020, 3, 15)) & (df['Date Posted'] <= datetime(2020, 3, 21)),
        (df['Date Posted'] >= datetime(2020, 3, 22)) & (df['Date Posted'] <= datetime(2020, 3, 28)),
        (df['Date Posted'] >= datetime(2020, 3, 29)) & (df['Date Posted'] <= datetime(2020, 4, 4)),
        (df['Date Posted'] >= datetime(2020, 4, 5)) & (df['Date Posted'] <= datetime(2020, 4, 11)),
        (df['Date Posted'] >= datetime(2020, 4, 12)) & (df['Date Posted'] <= datetime(2020, 4, 18)),
        (df['Date Posted'] >= datetime(2020, 4, 19)) & (df['Date Posted'] <= datetime(2020, 4, 25))]
    
    conditions_index = [
        (df['Date Posted'] >= datetime(2020, 3, 4)) & (df['Date Posted'] <= datetime(2020, 3, 7)),
        (df['Date Posted'] >= datetime(2020, 3, 8)) & (df['Date Posted'] <= datetime(2020, 3, 14)),
        (df['Date Posted'] >= datetime(2020, 3, 15)) & (df['Date Posted'] <= datetime(2020, 3, 21)),
        (df['Date Posted'] >= datetime(2020, 3, 22)) & (df['Date Posted'] <= datetime(2020, 3, 28)),
        (df['Date Posted'] >= datetime(2020, 3, 29)) & (df['Date Posted'] <= datetime(2020, 4, 4)),
        (df['Date Posted'] >= datetime(2020, 4, 5)) & (df['Date Posted'] <= datetime(2020, 4, 11)),
        (df['Date Posted'] >= datetime(2020, 4, 12)) & (df['Date Posted'] <= datetime(2020, 4, 18)),
        (df['Date Posted'] >= datetime(2020, 4, 19)) & (df['Date Posted'] <= datetime(2020, 4, 25))]
    
    
    
    choices_date = ['Mar 1 - Mar 7','Mar 8 - Mar 14', 'Mar 15 - Mar 21', 'Mar 22 - Mar 28', 'Mar 29 - Apr 4', 'Apr 5 - Apr 11', 'Apr 12 - Apr 18', 'Apr 19 - Apr 25']
    choices_index = [0, 1, 2, 3, 4, 5, 6, 7]

    df['Week Posted'] = np.select(conditions_date, choices_date)
    df['Week Index'] = np.select(conditions_index, choices_index)
    return df


def calculate_change (df):
    """Calculate the % change in job posting counts relatie to the first week"""
    
    df["% Change"] = np.where(df["Week Index"] != 1, df["Counts"]/df["Counts"][0] - 1, 0)
    df["% Change"] = round(df["% Change"].astype(float),3)
    return df


# Import dataset


In [38]:
df_usa = pd.read_csv('df_usa_april_week3_merged_no30days.csv')

df_usa['As of Date'] = pd.to_datetime(df_usa['As of Date'])
df_usa['Date Posted'] = pd.to_datetime(df_usa['Date Posted'])

df_usa.pipe(week_posted).head()

,Title,Location,Company,Rating,Salary,Sponsored,Date,As of Date,Description,Industry,Day Index,Date Posted,Week Posted,Week Index
0,Data Scientist Analyst,"Sunnyvale, CA",LOCKHEED MARTIN CORPORATION,4.0,None,None,2 days ago,2020-04-26,Perform exploratory research and analysis to i...,Datascience,2.0,2020-04-24,Apr 19 - Apr 25,7
1,"Data Science Intern (Summer, 2020)","Westfield Center, OH",Westfield Insurance,4.0,None,None,1 day ago,2020-04-26,"BUILD YOUR FUTURE, WHILE PROTECTING THEIRS.\nY...",Datascience,1.0,2020-04-25,Apr 19 - Apr 25,7
2,Data Science Intern,"San Diego, CA",HP,4.0,None,None,2 days ago,2020-04-26,Join us for a 12 week paid internship for the ...,Datascience,2.0,2020-04-24,Apr 19 - Apr 25,7
3,Data Scientist,Remote,Imperfect Foods,2.9,None,None,4 days ago,2020-04-26,About Imperfect\n\nImperfect Foods was founded...,Datascience,4.0,2020-04-22,Apr 19 - Apr 25,7
4,Data Scientist,"Richland, WA",Pacific Northwest National Laboratory,4.1,None,None,1 day ago,2020-04-26,Organization and Job ID\nJob ID: 310781\nDirec...,Datascience,1.0,2020-04-25,Apr 19 - Apr 25,7


## Graph Representations

In [39]:
# Exclude the very first week since it was not complete

counts_df_usa = df_usa[df_usa["Week Index"] != 0].groupby(['Week Index', 'Industry']).count()['Title'].reset_index()
counts_df_usa = counts_df_usa.rename(columns={'Title':'Counts'})
counts_df_usa = counts_df_usa.sort_values(["Industry", "Week Index"]).reset_index().drop(columns='index')

week_list=['Mar 8 - Mar 14', 'Mar 15 - Mar 21', 'Mar 22 - Mar 28', 'Mar 29 - Apr 4', 
           'Apr 5 - Apr 11', 'Apr 12 - Apr 18', 'Apr 19 - Apr 25']

#counts_df_usa

In [40]:
import plotly.graph_objects as go

title = 'USA: Job counts by week'
x = week_list
y_ds = counts_df_usa["Counts"][counts_df_usa["Industry"]=="Datascience"].values
y_en = counts_df_usa["Counts"][counts_df_usa["Industry"]=="Environmental"].values
y_ho = counts_df_usa["Counts"][counts_df_usa["Industry"]=="Hospitality"].values
y_he = counts_df_usa["Counts"][counts_df_usa["Industry"]=="Healthcare"].values
y_re = counts_df_usa["Counts"][counts_df_usa["Industry"]=="Retail"].values
y_fi = counts_df_usa["Counts"][counts_df_usa["Industry"]=="Finance"].values


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y_ds, name="Datascience",
                    line_shape='spline'))
fig.add_trace(go.Scatter(x=x, y=y_en, name="Environmental",
                    line_shape='spline'))
fig.add_trace(go.Scatter(x=x, y=y_ho, name="Hospitality",
                    line_shape='spline'))
fig.add_trace(go.Scatter(x=x, y=y_he, name="Healthcare",
                    line_shape='spline'))
fig.add_trace(go.Scatter(x=x, y=y_re, name="Retail",
                    line_shape='spline'))
fig.add_trace(go.Scatter(x=x, y=y_fi, name="Finance",
                    line_shape='spline'))


fig.update_traces(hoverinfo='text+name', mode='lines+markers')
fig.update_layout(title=title, legend=dict(y=0.5, traceorder='reversed', font_size=16))

fig.show()

In [63]:
### Calculate the change in job postings counts relative to the start and save results in df

industry = counts_df_usa["Industry"].unique()
counts_df_usa_change=pd.DataFrame()

for item in industry:
    df=counts_df_usa.copy()[counts_df_usa["Industry"]==item].reset_index().drop(columns='index')
    
    df.pipe(calculate_change)
        
    if counts_df_usa_change.empty:
        counts_df_usa_change=df
    else:
        counts_df_usa_change=counts_df_usa_change.append(df)


counts_df_usa_change=counts_df_usa_change.reset_index().drop(columns='index')


In [64]:
import plotly.graph_objects as go

title = 'USA: Change in job postings by week'
x = week_list
y_ds = counts_df_usa_change["% Change"][counts_df_usa_change["Industry"]=="Datascience"].values
y_en = counts_df_usa_change["% Change"][counts_df_usa_change["Industry"]=="Environmental"].values
y_ho = counts_df_usa_change["% Change"][counts_df_usa_change["Industry"]=="Hospitality"].values
y_he = counts_df_usa_change["% Change"][counts_df_usa_change["Industry"]=="Healthcare"].values
y_re = counts_df_usa_change["% Change"][counts_df_usa_change["Industry"]=="Retail"].values
y_fi = counts_df_usa_change["% Change"][counts_df_usa_change["Industry"]=="Finance"].values


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y_ds, name="Data Science",
                    line_shape='spline'))
#fig.add_trace(go.Scatter(x=x, y=y_en, name="Environmental",
#                    line_shape='spline'))
#fig.add_trace(go.Scatter(x=x, y=y_ho, name="Hospitality",
#                    line_shape='spline'))
fig.add_trace(go.Scatter(x=x, y=y_he, name="Healthcare",
                    line_shape='spline'))
#fig.add_trace(go.Scatter(x=x, y=y_re, name="Retail",
#                    line_shape='spline'))
fig.add_trace(go.Scatter(x=x, y=y_fi, name="Finance",
                    line_shape='spline'))

fig.add_trace(go.Scatter(
    x=["Mar 8 - Mar 14","Mar 8 - Mar 14"],
   y=[-.5, 2],
    name="Start of State of Emergency",
    
    line=dict(color="Purple", width=3, dash="dashdot")))

fig.update_traces(hoverinfo='text+name', mode='lines+markers')
fig.update_layout(yaxis_tickformat = '%', title=title, legend=dict(font_size=8),legend_orientation="h")


fig.show()